In [ ]:
크롤러: 크롤링 + 스크래핑
        ----- (*) => 얼마나 많은 웹페이지를 탐색
                ------- => 색인(어떤 정보가 어떤 페이지에 있었는지)
크롤링: 하이퍼링크를 따라서 계속해서 웹 공간을 탐색
        => 너비우선탐색, 깊이우선탐색, 한정된탐색(Focused) => depth, domain, element.tag, ... => 뉴스수집기
스크래핑: 어떤 정보(데이터)가 있는지를 추출/분석
          => HTML Parser(html.parser, lxml, html5lib)
          => 속성(id, class, arrts ...) + 구조
동적HTML(DHTML): 브라우저에 있는 DOM  //  네트워크.raw HTML 차이. 왜? 언제? 어떻게?

In [1]:
from requests import request
from requests.compat import urljoin

In [8]:
url = 'https://pythonscraping.com/pages/javascript/ajaxDemo.html'
resp = request('get', url)
# status_code, reason, headers

In [9]:
from bs4 import BeautifulSoup

In [10]:
dom = BeautifulSoup(resp.text, 'html.parser')
dom.body.script

<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>

In [11]:
dom.select_one('#content')

<div id="content">
This is some content that will appear on the page while it's loading. You don't care about scraping this.
</div>

In [12]:
import re

In [13]:
ajax = re.search(r'url:\s*"(.+?)"', dom.body.script.contents[0]).group(1)
nurl = urljoin(url, ajax)

In [14]:
resp = request('GET', nurl)
resp.status_code, resp.headers

(200,
 {'Server': 'nginx', 'Date': 'Tue, 18 Mar 2025 02:21:36 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '106', 'Connection': 'keep-alive', 'X-Powered-By': 'PHP/7.4.33, PleskLin', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'})

In [15]:
resp.text

'Here is some important text you want to retrieve! <p/><button id="loadedButton">A button to click!</button>'

In [16]:
dom.head

<head>
<title>Some JavaScript-loaded content</title>
<script src="../js/jquery-2.1.1.min.js"></script>
</head>

In [17]:
dom.body.script

<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>

In [18]:
url = 'https://brunch.co.kr/search'
params = {
    'q':'카리나',
    'type':'article'
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Whale/3.26.244.21 Safari/537.36'
}

resp = request('GET', url, params=params, headers=headers)
resp.status_code, resp.headers

(200,
 {'Date': 'Tue, 18 Mar 2025 02:21:59 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Content-Language': 'en-US', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Content-Encoding': 'gzip'})

In [19]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [20]:
dom.select_one('#resultArticle')

<div class="search_result" id="resultArticle"></div>

In [21]:
url = 'https://api.brunch.co.kr/v1/search/article'
params = {
    'q':'카리나',
    'page':'1',
    'pageSize':'20',
    'highlighter':'n',
    'escape':'n',
    'sortBy':'accu'
}
resp = request('GET', url, params=params, headers=headers)
resp.status_code, resp.headers['content-type']

(200, 'application/json;charset=UTF-8')

In [22]:
resp.request.url

'https://api.brunch.co.kr/v1/search/article?q=%EC%B9%B4%EB%A6%AC%EB%82%98&page=1&pageSize=20&highlighter=n&escape=n&sortBy=accu'

In [23]:
result = resp.json()

In [24]:
result['data'].keys()

dict_keys(['recommendProfiles', 'recommendProfileKeywords', 'totalPage', 'nextPage', 'query', 'list', 'totalCount', 'moreList', 'currentPage'])

In [25]:
result['data']['list'][0]

{'id': None,
 'version': None,
 'no': 137,
 'userId': 'eL95',
 'userName': None,
 'status': 'publish',
 'magazineNo': 0,
 'magazineAddress': None,
 'magazineAddressUri': None,
 'magazineArticleId': 0,
 'magazineTitle': None,
 'magazineStatus': None,
 'magazineType': None,
 'brunchbookSerialStatus': None,
 'magazineCoverImage': None,
 'commentWritable': False,
 'commentCount': 2,
 'donationCommentCount': 0,
 'donationCommentNoJson': None,
 'donationCommentList': None,
 'donationTotalAmount': 0,
 'likeCount': 0,
 'title': '카리나가 전하는 매운맛의 혁명! 스프라이트 광고 공개',
 'subTitle': None,
 'contentSummary': '카리나가 전하는 매운맛의 혁명! 스프라이트 광고 공개 <사진=코카콜라> 코카콜라가 세계 1위 사이다 브랜드 스프라이트의 새로운 광고 영상을 공개했습니다. 이번 광고는 걸그룹 에스파(aespa)의 리더 카리나와 손잡고 매운 음식을 더욱 특별하게 즐기는...',
 'contentHash': None,
 'authorComment': '작가의 한마디가 나옵니다.',
 'membershipContent': False,
 'promotionEnabled': False,
 'createTime': None,
 'updateTime': None,
 'publishTime': 1741094284000,
 'publishTimestamp': 1741094284000,
 'restrictTime': None,
 'content'

In [27]:
for row in result['data']['list']:
    print(row['title'])

카리나가 전하는 매운맛의 혁명! 스프라이트 광고 공개
파스쿠찌 - 카리나가 모든 걸 해결해주진 않는다
두 명의 카리나?! (ft. 파스쿠찌 x 카리나)
카리나가 전하는 크리스마스
이재욱, 입대 후 카리나와 결별 후유증?
“카리나만큼 예쁘다”…SM 하츠투하츠, 2월24일 데뷔
카리나 붕어빵
새로운 산타가 카리나? 신선하다.
공효진 절대 안 하는데, 카리나는 한다 "건강에 최악"
안나카리나
카리나
4세대 맥주 ‘크러시’- 카리나로 매출 46% up
안나카리나 장미 & 왁스핑크
'주술회전' 열풍, 에스파 카리나가 좋아한다고?
유지민 vs 카리나
'카리나의 굴욕'과 군중심리
카리나, 이강인, 류준열 그리고 대중
카리나 연애 무엇이 문제인가


In [28]:
for i in range(1,6):
    params['page'] = i
    resp = request('get', url, params=params, headers=headers)
    if re.search(r'application/json', resp.headers['content-type']):
        # if 'data' in resp.json()
        # if 'list' in resp.json()['data']
        for row in resp.json()['data']['list']:
            print(row['title'])

카리나가 전하는 매운맛의 혁명! 스프라이트 광고 공개
파스쿠찌 - 카리나가 모든 걸 해결해주진 않는다
두 명의 카리나?! (ft. 파스쿠찌 x 카리나)
카리나가 전하는 크리스마스
이재욱, 입대 후 카리나와 결별 후유증?
“카리나만큼 예쁘다”…SM 하츠투하츠, 2월24일 데뷔
카리나 붕어빵
새로운 산타가 카리나? 신선하다.
공효진 절대 안 하는데, 카리나는 한다 "건강에 최악"
안나카리나
카리나
4세대 맥주 ‘크러시’- 카리나로 매출 46% up
안나카리나 장미 & 왁스핑크
'주술회전' 열풍, 에스파 카리나가 좋아한다고?
유지민 vs 카리나
'카리나의 굴욕'과 군중심리
카리나, 이강인, 류준열 그리고 대중
카리나 연애 무엇이 문제인가
"결국 열애설 사과문 올렸다" 카리나, 해외 팬들→비난
카리나 열애설... 손가락질 받아야 할 대상은?
"어떻게든 만났을 운명" 이재욱, ♥카리나와 여행예능?
이재욱 카리나 열애설에 '환승연애설'까지
차은우는 왜 차은우이고 카리나는 왜 카리나일까?
[카리나]의 출연을 상상하며 만들어 보는 캐릭터
누구나 설득되는 퇴사사유 3가지
여전히 종이에 필기하시는 분 없나요
카리나 3행시 할게요 #에스파 #aespa
카리나 3행시 할게요 #에스파 #aespa
카리나가 선택한 공항패션 - Weekly Celeb
목도리가 지어줄 작은 집
이것도 퇴사 사유가 되나요? (1) 거리
주씨, 후레쉬 11
남자들이 쉽게 꼬이는 여자들의 '공통된 행동' 7가지
in Drop or Go] 3월 1주차
귀신과 성관계, 인큐버스(incubus)와 무의식
"너 향기 되게 좋다"…향수 제대로 뿌리는 방법 5가지
나의 추구미는...
14편 - 미스터리 수사단
밀라노패션위크 2025 F/W – 프라다
아이돌 덕질로 시작하는 운명 체인지 주문
착하다는 상위 가치는 나의 착한 남자일 때 의미가 있죠
지드래곤 TOO BAD는 TOO GOOD
이제 TV 브랜드 광고의 시대는 갔다
백화점 3사 크리스마스 마켓, 한번 비교해 봤습니다
나는 어디서 왔나 오 에
난 이 콘서트를 4번 

In [29]:
url = 'https://brunch.co.kr/'
resp = request('GET', url, headers=headers)
dom = BeautifulSoup(resp.text, 'html.parser')

In [30]:
for el in dom.select('form input[name]'):
    print(el.attrs['name'], el.attrs['value'])


q 
profileId 


In [31]:
params['q']

'카리나'

In [32]:
dom.select_one('form').attrs

{'action': '/search', 'method': 'get'}

In [33]:
url = 'https://api.brunch.co.kr/v1/search/article'

In [34]:
url = 'https://brunch.co.kr/search'
resp = request('GET', url, headers=headers)
dom = BeautifulSoup(resp.text, 'html.parser')

In [35]:
for el in dom.select('form > input[name]'):
    print(el.attrs['name'], el.attrs['value'])

q 
profileId 
q 
type article


In [36]:
dom.select_one('form').attrs

{'action': '/search', 'method': 'get'}

In [ ]:
Client           /search           /api
DOM
[form]-----------> GET
 <input name='' value=''> => ?name=value
                    JS ------------> GET
                           AJAX
                       <------------ JSON
DHTML <-------------

In [ ]:
HTML ---(요청)---->
     FORM   action=>주소
     INPUT  name,value=>?name=value
DOM  <------------

<script src=".js">
<script>
 $.ajax();
</script>  => JS 해석 X, 브라우저(개발자도구)-네트워크탭

JS(XHR) ---(요청)---> 서버
        <----------- content-type
Data(DOM을 새롭게 그리지 X) => 해석

In [37]:
url = 'https://api.brunch.co.kr/v1/search/live'
params = {'q':None}
params['q'] = '카리나'
resp = request('get', url, params=params, headers=headers)
for a in resp.json()['data']['article']:
    print(a['title'])
# 'magazine' in resp.json()['data'].keys()
# 'article'
# filter(lambda k:re.match('magazine|article'), resp.json()['data'].keys())

<b>카리나</b>가 전하는 매운맛의 혁명! 스프라이트 광고 공개
<b>카리나</b>
파스쿠찌 - <b>카리나</b>가 모든 걸 해결해주진 않는다
두 명의 <b>카리나</b>?! (ft. 파스쿠찌 x <b>카리나</b>)
<b>카리나</b>가 전하는 크리스마스
이재욱, 입대 후 <b>카리나</b>와 결별 후유증?
“<b>카리나</b>만큼 예쁘다”…SM 하츠투하츠, 2월24일 데뷔


In [ ]:
# 네이버 추천검색어
url = 'https://ac.search.naver.com/nx/ac'
params = {
    'q':None,
    'con':'0',
    'frm':'nv',
    'ans':'2',
    'r_format':'json',
    'r_enc':'UTF-8',
    'r_unicode':'0',
    't_koreng':'1',
    'run':'2',
    'rev':'4',
    'q_enc':'UTF-8',
    'st':'100'
}

while True:
    query = input()
    if query == 'q':
        break

    params['q'] = query
    resp = request('GET', url, params=params, headers=headers)
    if re.search(r'application/json', resp.headers['content-type']):
        print(resp.json()['items'])

In [ ]:
Client           Server
      ----------->
      <----------- callback1()
$.ajax({success:func(response) 어떻게...})
func callback1()
func callback2()
func callback3()

In [ ]:
# 네이버웹툰
1단계
[WEB]  https://comic.naver.com/index ==> DOM1 (div#root)
[AJAX] https://comic.naver.com/api/home/component?type=DAILY_WEBTOON&order=USER
       titleList: 
         0: {titleId: 783053, titleName: "김부장", ...}

2단계
[WEB]  https://comic.naver.com/webtoon/list?titleId=783053 ==> DOM2 (div#root)
[AJAX] https://comic.naver.com/api/article/list?titleId=783053&page=1&sort=DESC
       articleList: 
         0: {no: 177, ...}
3단계
[WEB]  https://comic.naver.com/webtoon/detail?titleId=783053&no=177&week=tue ==> DOM3 (#sectionContWide)

4단계
[IMG]  url -> 저장

In [ ]:
URLs = ['https://comic.naver.com/api/home/component?type=DAILY_WEBTOON&order=USER']
Seens = []

while URLs:
    seed = URLs.pop(0)
    Seens.append(seed)

    resp = request('GET', seed, headers=headers)
    if resp.status_code != 200:
        continue

    # 4단계
    if re.search(r'image/jpeg', resp.headers['content-type']):
        file = re.search(r'.+/(.+[.]jpg)$', seed).group(1)
        with open(file, 'wb') as fp:
            fp.write(resp.content)

    # 3단계 
    if re.search(r'text/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        for img in dom.select('#sectionContWide img[src]'):
            nurl = img.attrs['src']
            if nurl not in URLs and nurl not in Seens:
                URLs.append(nurl)

    if not re.search(r'application/json', resp.headers['content-type']):
        continue

    result = resp.json()

    # 웹툰별 목록
    if 'titleList' in result.keys():
        # 1단계
        for r in result['titleList']:
            url = 'https://comic.naver.com/api/article/list'
            params = {
                'titleId':r['titleId'],
                'page':'1',
                'sort':'DESC'
            }
            nurl = url+'?'+urlencode(params)
            if nurl not in URLs and nurl not in Seens:
                URLs.append(nurl)
                break

    # 특정 웹툰의 회차 목록
    if 'articleList' in result.keys():
        # 2단계
        for r in result['articleList']:
            url = 'https://comic.naver.com/webtoon/detail'
            params = {
                'titleId':re.search(r'titleId=(\d+)&', seed).group(1),
                'no':r['no'],
                'week':'tue'
            }
            nurl = url+'?'+urlencode(params)
            if nurl not in URLs and nurl not in Seens:
                URLs.append(nurl)
                break

In [ ]:
from requests.compat import urlencode